# 고객의 자동차 재구매 경향 분석
* '대차'와 '추가구매'로 분류
* '구매이력'과 '차량 보유정보' 등을 활용해 재구매 유형별 구매 시점 예측

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# 과제 1

## 01. '대차'와 '추가구매' 분리/추정
로직 개발

In [3]:
cars = pd.read_csv("/content/drive/MyDrive/해카톤/'21년 해카톤_데이터/'21년 해카톤_차량정보.csv", encoding='cp949')
cars.head()

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM
0,H1308052741CBFA022907,AOZZZZO010000035,20140627,20140627,20170831.0,LF쏘나타,승용,중형,가솔린 2.0 CVVL,SMART
1,H1308050341LBEA656859,AOZZZZO010000067,20140711,20140711,NaN,YF쏘나타,승용,중형,LPG 2.0,PREMIUM
2,H1308052741DBFA038433,AOZZZZO010000190,20140707,20140707,NaN,LF쏘나타,승용,중형,LPi 2.0,PREMIUM
3,H13080607413BFA007670,AOZZZZO010000371,20140711,20140711,NaN,그랜저HG,승용,대형,하이브리드,PREMIUM
4,H1308052741DBFA091812,AOZZZZO010000492,20150120,20150120,NaN,LF쏘나타,승용,중형,LPi 2.0,PREMIUM


In [4]:
cars1 = cars[['CAR_ID','CUS_ID','WHOT_DT','CAR_HLDG_FNH_DT','CAR_NM']]
cars1.head()

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM
0,H1308052741CBFA022907,AOZZZZO010000035,20140627,20170831.0,LF쏘나타
1,H1308050341LBEA656859,AOZZZZO010000067,20140711,NaN,YF쏘나타
2,H1308052741DBFA038433,AOZZZZO010000190,20140707,NaN,LF쏘나타
3,H13080607413BFA007670,AOZZZZO010000371,20140711,NaN,그랜저HG
4,H1308052741DBFA091812,AOZZZZO010000492,20150120,NaN,LF쏘나타


In [5]:
cars1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835830 entries, 0 to 1835829
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   CAR_ID           object 
 1   CUS_ID           object 
 2   WHOT_DT          int64  
 3   CAR_HLDG_FNH_DT  float64
 4   CAR_NM           object 
dtypes: float64(1), int64(1), object(3)
memory usage: 70.0+ MB


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# 날짜 데이터 타입 변경(-> str -> datetime)
cars1['WHOT_DT'] = cars1['WHOT_DT'].astype('str')
cars1['WHOT_DT'] = pd.to_datetime(cars1['WHOT_DT'])

# cars1['CAR_HLDG_STRT_DT'] = cars1['CAR_HLDG_STRT_DT'].astype('str')
# cars1['CAR_HLDG_STRT_DT'] = pd.to_datetime(cars1['CAR_HLDG_STRT_DT'])

# float64 타입에다가 NaN 값까지 가지고 있는 컬럼 처리
cars1['CAR_HLDG_FNH_DT'][cars1['CAR_HLDG_FNH_DT'].isnull()] = '20211231' # 임의로 먼 미래의 날짜로 지정하려했지만, pandas에서 사용 불가 오류로 이번 해 마지막 날짜로 지정
cars1['CAR_HLDG_FNH_DT'] = cars1['CAR_HLDG_FNH_DT'].astype('int')
cars1['CAR_HLDG_FNH_DT'] = cars1['CAR_HLDG_FNH_DT'].astype('str')
cars1['CAR_HLDG_FNH_DT'] = pd.to_datetime(cars1['CAR_HLDG_FNH_DT'])

In [8]:
# 차량ID, 고객ID, 출고일자, 보유종료일자, 차명
cars2 = cars1.sort_values(['CUS_ID', 'WHOT_DT', 'CAR_HLDG_FNH_DT'], ignore_index=True)
cars2

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2007-07-12,베르나
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2021-12-31,그랜저HG
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2021-12-31,쏘나타
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2021-12-31,제네시스
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2011-06-08,싼타페
...,...,...,...,...,...
1835825,H1308051441BP2A665825,ATZZZZV310001228,2002-04-17,2004-11-09,쏘나타
1835826,H1308030751BPYU060804,ATZZZZV310001232,2000-05-31,2008-04-12,베르나
1835827,H1308040741LBCU575856,ATZZZZV310001232,2012-06-15,2014-12-06,아반떼MD
1835828,H1308051441SPYA320981,ATZZZZV310001236,2000-06-09,2004-02-04,쏘나타


### '대차'와 '추가구매'

* 대차: *기존 보유하던 차량을 처분하고, 신차를 출고하는 경우*
* 추가구매: *기존 보유하던 차량을 처분하지 않고, 추가로 신차를 출고하는 경우*

### 분리 기준
* 신차구매 = 출고날짜 기준
* 처분 시점 = 보유 종료 일자 기준
* 첫 번째 구매 차량은 '첫구매'로 가정
* **기존 보유 차량의 처분 시점 전후 3개월 내 신차 구매를 '대차'로 가정**
* **대차의 경우 제외 후, 기존 보유 차량 이외로 추가 신차 구매를 '추가구매'로 가정**

In [9]:
cars2['CUS_ID'].value_counts(sort=True, dropna=True)

ATZZTZT260000436    12
AONNHZF030000491    12
ATZZVZE050001085    12
ATZZTZH040001065    12
ATZZFZN210000948    12
                    ..
ATZOFOZ130001259     1
ATZZSZE160001760     1
ATZZZZT180000816     1
ATZZPZE180000337     1
ATZOVZO130000438     1
Name: CUS_ID, Length: 1096206, dtype: int64

In [10]:
cars2[cars2['CUS_ID']=='AONNHZF030000491'] # 여러 대의 차를 한꺼번에 소유하면서 거래를 한 경우도 존재

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM
25113,H1308190481VP1U100413,AONNHZF030000491,2001-05-12,2007-07-19,싼타페
25114,H1308030741FP2U369354,AONNHZF030000491,2002-04-17,2007-03-16,베르나
25115,H1308030741FP2U389421,AONNHZF030000491,2002-06-20,2021-12-31,베르나
25116,H1308030741FP2U432002,AONNHZF030000491,2002-10-22,2004-05-12,베르나
25117,H1308190481VP4U738510,AONNHZF030000491,2004-03-17,2007-03-06,싼타페
25118,H1308042141BP7U148283,AONNHZF030000491,2007-02-12,2009-12-04,아반떼
25119,H1308022151DP7U723866,AONNHZF030000491,2007-09-05,2009-12-08,클릭
25120,H1308192381UBDU169069,AONNHZF030000491,2013-05-20,2017-08-17,싼타페 DM
25121,H1308040741DBFU360270,AONNHZF030000491,2014-12-09,2021-12-31,아반떼MD
25122,H1308042741DBHU278824,AONNHZF030000491,2016-09-06,2021-12-31,아반떼AD


In [11]:
# 차량 구매 특징을 구분할 새로운 컬럼 생성 'TYPE'
cars2['TYPE'] = '미정'

# 현재까지 소유한 차량 '소유'로 표기
# cars2['TYPE'][cars2['CAR_HLDG_FNH_DT']=='2021-12-31'] = '소유'

In [12]:
cars2.head()

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM,TYPE
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2007-07-12,베르나,미정
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2021-12-31,그랜저HG,미정
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2021-12-31,쏘나타,미정
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2021-12-31,제네시스,미정
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2011-06-08,싼타페,미정


In [13]:
# 일부 데이터 먼저 활용
cars3 = cars2[:300]

In [14]:
cars3.head()

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM,TYPE
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2007-07-12,베르나,미정
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2021-12-31,그랜저HG,미정
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2021-12-31,쏘나타,미정
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2021-12-31,제네시스,미정
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2011-06-08,싼타페,미정


In [15]:
# 회원별로 데이터프레임을 만들어서 처리 후 다시 합치기
df = []

for customer in cars3['CUS_ID'].unique():
  temp = cars3[cars3['CUS_ID']==customer]
  temp.reset_index(inplace=True, drop=True)
  temp.sort_values(by='WHOT_DT', ascending=False)

  # 첫 번째 구매 차량 '첫구매'로 표기
  temp['TYPE'][0] = '첫구매'

  for start in range(1,len(temp)):
    for end in range(len(temp)):
      # 3개월 이내에 기존 차량 처분 후 새차량 구매한 경우
      if start > end:
        if (temp['CAR_HLDG_FNH_DT'][end] - temp['WHOT_DT'][start]).days > 0:
          temp['TYPE'][start] = '추가구매'
      if start != end:
        if 0 < (temp['CAR_HLDG_FNH_DT'][end] - temp['WHOT_DT'][start]).days < 90:
          temp['TYPE'][start] = '대차'
      
  df.append(temp)


df = pd.concat(df)
df.reset_index(inplace=True, drop=True)

* '미정'은 기존 보유 차량 처분 후 3개월 이상 차량을 미소지한 후 새차량을 구매한 경우

In [16]:
df.head(40)

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_FNH_DT,CAR_NM,TYPE
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2007-07-12,베르나,첫구매
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2021-12-31,그랜저HG,미정
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2021-12-31,쏘나타,첫구매
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2021-12-31,제네시스,추가구매
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2011-06-08,싼타페,첫구매
5,H1308192381UBDU081735,AONEEOO020000170,2012-11-20,2021-12-31,싼타페 DM,미정
6,H1308191081UBBU647205,AONEEOO020000290,2010-08-26,2011-10-28,싼타페,첫구매
7,H1308192381UBJU777377,AONEEOO020000290,2017-04-06,2021-12-31,싼타페 DM,미정
8,H1308190881WP6U064927,AONEEOO040000233,2006-06-16,2017-05-30,싼타페,첫구매
9,H1308042141BBAU837494,AONEEOO040000233,2009-07-27,2021-12-31,아반떼,추가구매


In [ ]:
# 전체 데이터 대입

# 회원별로 데이터프레임을 만들어서 처리 후 다시 합치기
df2 = []

for customer in cars2['CUS_ID'].unique():
  temp = cars2[cars2['CUS_ID']==customer]
  temp.reset_index(inplace=True, drop=True)
  temp.sort_values(by='WHOT_DT', ascending=False)

  # 첫 번째 구매 차량 '첫구매'로 표기
  temp['TYPE'][0] = '첫구매'

  for start in range(1,len(temp)):
    for end in range(len(temp)):
      # 3개월 이내에 기존 차량 처분 후 새차량 구매한 경우
      if start > end:
        if (temp['CAR_HLDG_FNH_DT'][end] - temp['WHOT_DT'][start]).days > 0:
          temp['TYPE'][start] = '추가구매'
      if start != end:
        if 0 < (temp['CAR_HLDG_FNH_DT'][end] - temp['WHOT_DT'][start]).days < 90:
          temp['TYPE'][start] = '대차'
      
  df2.append(temp)


df2 = pd.concat(df2)
df2.reset_index(inplace=True, drop=True)

In [ ]:
df2.head(30)

In [ ]:
df2.to_excel('./재구매유형 추가 데이터.xlsx')

## 02. EDA
고객과 차량 관점에서 특징과 차이점 도출